# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testing part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file `housing_features_description.txt` for more information on the attributes.


```python
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### NOTE: The boston dataset has an ethical problem. More on this can e.g. be found in the scikit documentation. ###
### Summary: The dataset contains the proportion of black people, which was assumed that racial self-segregation had a positive impact on house prices. ###


data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
boston_data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
boston_target = raw_df.values[1::2, 2]

df=pd.DataFrame(boston_data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston_data
y=boston_target

```


```python
#df.tail()
#print(X)
#print(boston_target)
#print(boston_data)
```

### Note - attention

Think about the ethical aspects of this dataset and machine learning in general. 

Can you always trust your data source? Can we use every possible information for our models?

## Exercise 1

Write the `split_train_test`$({\color{RawSienna}X},{\color{Dandelion}y},{\color{Tomato}\mathrm{split}},{\color{Tomato}\mathrm{seed}})$, given an instance matrix ${\color{RawSienna}X} \in {\color{Tan}\mathbb{R}}^{N \times D}$, labels ${\color{Dandelion}y} \in {\color{Tan}Y}^N$, a split ratio ${\color{Tomato}\mathrm{split}} \in [0, 1]$ and a random seed ${\color{Tomato}\mathrm{seed}} \in {\color{Tan}\mathbb{Z}}$. Split the dataset in $({\color{Tomato}\mathrm{split}}×100)\%$ of the instances for training our model and the rest for testing, i.e. 

$$ \left|{\color{RawSienna}X}_{\text{train}}\right| = \lceil {\color{Tomato}\text{split}} \cdot N \rceil, \qquad |{\color{RawSienna}X}_{\text{train}}| + |{\color{RawSienna}X}_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- `X_train`$={\color{RawSienna}X}_{\text{train}}$, `y_train`$={\color{Dandelion}y}_{\text{train}}$: the training instances and labels;
- `X_test`$={\color{RawSienna}X}_{\text{test}}$, `y_test`$={\color{Dandelion}y}_{\text{test}}$: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. `np.random.shuffle`).


```python
#np.random.default_rng(100)
#np.random.shuffle(X)
def split_train_test(X, y, split, seed):
    k = int(split*len(y))
    np.random.default_rng(seed)
    np.random.shuffle(X)
    X_train, X_test = X[:k,:], X[k:,:]
    y_train, y_test = y[:k], y[k:]
    return X_train, y_train, X_test, y_test
```


```python
#X_train, y_train, X_test, y_test = split_train_test(X, y, 0.5, 200)
```

## Exercise 2

Write the function `train_linear_reg`$({\color{RawSienna}X}_{\text{train}}, {\color{Dandelion}y}_{\text{train}}, {\color{Tomato}\lambda})$.

Implement the ridge regression model (slide 24). The function should output the learned weight vector ${\color{Tomato}\theta} \in {\color{Tan}\mathbb{R}}^D$ or ${\color{Tan}\mathbb{R}}^{D+1}$ (depending on whether you are adding **bias**).

Calculation of the weight vectors ${\color{Tomato}\theta}$:

$$
{\color{Tomato}\theta}=({\color{RawSienna}X}^{\color{CornflowerBlue}\mathrm{T}}{\color{RawSienna}X}+{\color{Tomato}\lambda}I)^{-1}{\color{RawSienna}X}^{\color{CornflowerBlue}\mathrm{T}}{\color{Dandelion}y}
$$


```python
#lmbd=1.
#np.dot( np.linalg.inv(np.dot(X_train.T,X_train)+lmbd*np.identity(len(X_train.T))), np.dot(X_train.T, y_train))
def train_linear_reg(X, y, lbda):
    return np.dot( np.linalg.inv(np.dot(X.T, X) + lbda*np.identity(len(X.T))), np.dot(X.T, y) ) # theta
```


```python
#train_linear_reg(X_train, y_train, 1)
#lbda*np.identity(len(X.T))
```

## Exercise 3

Write the function `predict`$({\color{RawSienna}X},{\color{Tomato}\theta})$ which predicts housing values vector pred for a dataset ${\color{RawSienna}X}$ and a previously trained parameter vector ${\color{Tomato}\theta}$.


```python
def func(theta, X):
    return np.dot(X, theta)

def predict_lin(X, theta):
    #numberofpredictions=0
    pred=np.zeros(len(X))
    for i in range(len(X)):
        pred[i]=func(theta, X[i])
    return pred # y_pred
```

## Exercise 4

Write the function `mean_abs_loss`$({\color{Dandelion}y}_{\text{true}}, {\color{Dandelion}y}_{\text{pred}})$ which computes the mean of the absolute differences between our prediction vector ${\color{Dandelion}y}_{\text{pred}}$ and the real housing values ${\color{Dandelion}y}_{\text{true}}$.


```python
def mean_abs_loss(y_true, y_pred):
    mean=0
    for i in range(len(y_true)):
        mean+=abs(y_true[i]-np.round(y_pred[i], decimals=1))
    mean=mean/len(y_true)
    return mean
```

## Exercise 5

Evaluate your solutions by running the following code. 

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?


```python
seed = 3
lbda = 0.5
split = 0.7
X_train, y_train, X_test, y_test = split_train_test(X,y,split,seed)

theta = train_linear_reg(X_train, y_train, lbda)

y_pred = predict_lin(X_test, theta)

mae = mean_abs_loss(y_test, y_pred)

print(theta)

print(y_test.mean())

print(np.round(y_pred, decimals=1))
print(y_test)
print(mae)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae*1000))
```